# Decentralized Prediction

## Summary

Here you'll find a usage of the decentralized prediction implementation.
You have a py file that do the same avaialble: decentralized.py

The notebook will use the User and Server classes that are used for simulate the network of agents that wants to create a similarity matrix without giving up data.
These classes are documented in the class files

In [1]:
import numpy as np
import pandas as pd
import os
from model.track_collection import TrackCollection
from utils.collection_splitter import splitter
from agent.server import Server
from agent.user import User

## Configuration

We created a way to extract rating of each track from a user's library. 
The probleme we faced is that we just have one real user's library. So we can't use it for the prediction (we need more users, and if we split the lib of this user in multiple libraries, it'll not be relevant because the rating will be the same).
So we created a totally fake music library, and 5 users with a part of the global library and notation on it. 
You can find the details in files:
* data/track_collection_test.json - The global library
* data/users/i.json - The lib of the ith user

So here in the config, the commented code is dynamic, but not relevant as we have just one real library. And the other code is hardcorded for the 5 test users.

In [2]:
#### CONFIG 
number_of_users = 5

### Loading the tracks data; and splitting them into number_of_users collections

track_collection = TrackCollection()
track_collection.load(os.path.join('data', 'track_collection_test.json'))
df_track_collection = track_collection.to_dataframe()
track_list = df_track_collection[['id']]

#user_collections = splitter(track_collection, number_of_users, 0.3)
#user_dfs = []

### Generating the users_dataframes vector with all tracks and their ratings

#for user_collection in user_collections:
#  ndf = user_collection.to_dataframe()[['id', 'rating_score']]
#  user_matrix = track_list.merge(ndf, on='id', how='left').fillna(0)
#  user_dfs.append(user_matrix[['rating_score']])


user_dfs = []
for i in range(number_of_users):
  tc = TrackCollection()
  tc.load(os.path.join('data', 'users', str(i+1)+'.json'))
  ndf = tc.to_dataframe()[['id', 'rating_score']]
  user_matrix = track_list.merge(ndf, on='id', how='left')
  user_dfs.append(user_matrix[['rating_score']])

### Generating the user
users = []
i = 0
for df in user_dfs:
  users.append(User(i, df))
  i += 1

### Setting the user loop: each user have to know which one is the next, in order to compute the decentralized calculuses
for user in users:
  if(user.id < number_of_users-1):
    user.nextNode = users[user.id+1]
  else:
    user.nextNode = users[0]



## Runing the server

Now we have created all the users, and created a loop of users, we can create the server and run it.
By runing it, it'll generate the similmarity matrix and spread it to all users.

So at the end of this executio, all users will have the similarity matrix calculated in a decentralized way.

In [3]:
### Generating the server
server = Server(users, track_list)

### Running the server
server.run()

### Printing the similarity matrix
users[0].similarity_matrix


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.587975,-inf,0.000000,0.000000,0.070711,-0.070711,-0.070711,-0.424264,-0.388615,0.000000,0.240887,-0.565685,-0.565685,0.000000,0.494975,-0.535262,-0.083563,0.081650,0.141421
1,0.587975,1.000000,-inf,0.000000,0.000000,-0.188982,0.188982,-0.377964,-0.566947,-0.842651,0.000000,-0.245256,0.000000,0.000000,0.000000,-0.377964,-0.921038,0.322590,0.545545,0.188982
2,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
3,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.070711,-0.188982,-inf,0.000000,0.000000,1.000000,-1.000000,0.500000,0.000000,0.570323,0.000000,-0.648886,0.000000,0.000000,0.000000,0.500000,0.362933,-0.919145,0.577350,0.500000
6,-0.070711,0.188982,-inf,0.000000,0.000000,-1.000000,1.000000,-0.500000,0.000000,-0.570323,0.000000,0.648886,0.000000,0.000000,0.000000,-0.500000,-0.362933,0.919145,-0.577350,-0.500000
7,-0.070711,-0.377964,-inf,0.000000,0.000000,0.500000,-0.500000,1.000000,-0.500000,0.777714,0.000000,-0.081111,0.500000,0.500000,0.000000,0.500000,0.155543,-0.787839,0.000000,-0.500000
8,-0.424264,-0.566947,-inf,0.000000,0.000000,0.000000,0.000000,-0.500000,1.000000,0.155543,0.000000,0.081111,-0.500000,-0.500000,0.000000,0.000000,0.777714,0.196960,-0.288675,0.500000
9,-0.388615,-0.842651,-inf,0.000000,0.000000,0.570323,-0.570323,0.777714,0.155543,1.000000,0.000000,-0.033643,0.207390,0.207390,0.000000,0.570323,0.741935,-0.755679,-0.209540,-0.207390


## Prediction

Now the similarity matrix is created, we can do the predictions.
The predictions can be computed locally for an user. That's perfect: this way we don't give any inforamtion to other users

In [9]:

### The list of notes that users don't have in their library
userToPredict = []
userToPredict.append([2,3,4,5,6,10,11,14,15,18])
userToPredict.append([0,2,3,8,12,13,14,18,19])
userToPredict.append([2,3,4,5,6,7,10,12,13,14,15,17,19])
userToPredict.append([2,3,7,8,9,10,12,13,14,15,16])
userToPredict.append([1,2,4,5,6,7,8,9,10])

i = 0
for uToPredict in userToPredict:
    for j in uToPredict:
        if users[i].willILikeIt(j):
            print("User %d will probably like song %i " % (i,j))
        #else:
        #    print("User %d will probably not like song %i. Because the predicted score (%f) to low (av: %f)" % (i,j,users[i].predict_rating(j), users[0].average_rating))
    i+=1


User 0 will probably like song 15 
User 1 will probably like song 0 
User 1 will probably like song 18 
User 2 will probably like song 5 
User 2 will probably like song 6 
User 2 will probably like song 7 
User 2 will probably like song 12 
User 2 will probably like song 13 
User 2 will probably like song 15 
User 3 will probably like song 7 
User 3 will probably like song 8 
User 3 will probably like song 9 
User 3 will probably like song 12 
User 3 will probably like song 13 
User 3 will probably like song 15 
User 3 will probably like song 16 
User 4 will probably like song 5 
User 4 will probably like song 6 
User 4 will probably like song 9 


And we have some predictions ! **YEAH !**